In [18]:
import warnings
import xarray as xr
from satpy import Scene
from datetime import datetime
import multiprocessing
import os
import pathlib
import sys
from datetime import datetime
from functools import partial
from typing import Optional

import autoroot  # needed for imports from src
import numpy as np
import tqdm
import xarray as xr
from loguru import logger
from scipy.interpolate import InterpolatedUnivariateSpline
from sklearn.neighbors import BallTree
from sklearn.utils.extmath import weighted_mode

#### STEPS (following Will's code)

- load msg invariants?
- get satellite viewing angle 
- correct parallax

In [4]:
def load_msg_file(
    filename: str, variables: list[str]
) -> xr.Dataset:
    """preprocessing.preprocessing_utils.msg_utils.load_msg_native_from_storage

    Download and load an MSG SEVIRI native file from GCP as an xarray dataset

    Parameters
    ----------
    filename : str
        Path of SEVIRI file to load on GCP
    variables : list[str], optional
        List of SEVIRI variables to load, by default ALL_MSG_VARS

    Returns
    -------
    xr.Dataset
        Dataset of SEVIRI image, calibrated to radiance
    """
    scn = Scene(filenames=[filename], reader="seviri_l1b_native")
    if variables:
        scn.load(variables, calibration="radiance")
    msg_ds = scn.to_xarray().drop_vars("msg_seviri_fes_3km", errors="ignore")
    return msg_ds

In [20]:
def get_msg_datetime_from_filename(filename: str) -> datetime:
    """preprocessing.preprocessing_utils.msg_utils.get_msg_datetime_from_filename

    Get a datetime representation of an MSG SEVIRI image end time from its
    filename

    Parameters
    ----------
    filename : str
        Filename to parse

    Returns
    -------
    datetime
        End date of file
    """
    file_datestr = filename.split("/")[-1].split(".")[0][-14:]
    dtime = datetime.strptime(file_datestr, "%Y%m%d%H%M%S")
    return np.datetime64(dtime.isoformat())

In [21]:
def get_satellite_viewing_angles(
    lat: float,
    lon: float,
    sat_lat: float = 0,
    sat_lon: float = 0,
    sat_alt: float = 35_793,
) -> tuple[float, float]:
    """Calculate satellite zenith and azimuth angles

    Parameters
    ----------
    lat : float
        latitude of surface point in degrees
    lon : float
        longitude of surface point in degrees
    sat_lat : float, optional
        latitude of sub-satellite point in degrees, by default 0
    sat_lon : float, optional
        longitude of sub-satellite point in degrees, by default 0
    sat_alt : float, optional
        altitude of satellite in km, by default 35_793 (geostationary orbit
        height over average earth radius)

    Returns
    -------
    tuple[float, float]
        satellite zenith and azimuth angles in degrees
    """

    with warnings.catch_warnings():
        # Ignore warnings caused by non-finite lat/lon values
        warnings.simplefilter("ignore")

        Re = 6_371
        Rgeo = sat_alt + Re

        # Caclulate the beta angle
        cos_beta = np.cos(np.radians(lat - sat_lat)) * np.cos(np.radians(lon - sat_lon))
        sin_beta = np.sin(np.arccos(cos_beta))

        # Calculate satellite zenith angle
        geo_dist = (
            Rgeo**2 + Re**2 - 2 * Rgeo * Re * cos_beta
        ) ** 0.5  # distance from surface to satellite
        sin_theta = (Rgeo * sin_beta) / geo_dist
        zenith_angle = np.degrees(np.arcsin(sin_theta))
        # Find where satellite-surface path intersects the earth and make these > 90
        zenith_angle = np.where(
            geo_dist**2 < (Rgeo**2 - Re**2), zenith_angle, 180 - zenith_angle
        )

        # Calculate satellite azimuthal angle
        x_sat = np.cos(np.radians(lat - sat_lat)) * np.sin(np.radians(lon - sat_lon))
        y_sat = np.sin(np.radians(lat - sat_lat))
        azimuth_angle = np.where(
            np.isfinite(x_sat), np.degrees(np.arctan2(x_sat, y_sat)) % 360, np.nan
        )

    return zenith_angle, azimuth_angle

In [22]:
def create_msg_invariants() -> None:
    """preprocessing.preprocessing_utils.msg_utils.create_msg_invariants

    Create a netcdf file of invariant properties of SEVIRI images (latitude,
    longitude, zenith and azimuthal values and pixel time offsets)
    """
    example_file = '/home/anna.jungbluth/data/MSG2-SEVI-MSG15-0100-NA-20200113095743.406000000Z-NA.nat'

    invariants_ds = load_msg_file(example_file, variables=["IR_108"]
    ).drop_vars("IR_108")

    # Add zenith angles and azimuth angles
    zenith_angles, azimuth_angles = get_satellite_viewing_angles(
        invariants_ds.latitude.values, invariants_ds.longitude.values
    )
    invariants_ds["satellite_zenith_angle"] = (("y", "x"), zenith_angles)
    invariants_ds["satellite_azimuth_angle"] = (("y", "x"), azimuth_angles)

    # Add time offsets
    file_time = get_msg_datetime_from_filename(example_file)

    pixel_time_offsets = invariants_ds.acq_time - file_time
    invariants_ds["time_offsets"] = pixel_time_offsets

    invariants_ds.to_netcdf("msg_invariants.nc")

In [ ]:
# TODO: Finish these functions

def correct_for_parallax(self):
        """Correct cloudsat lats/lons for parallax when comparing to SEVIRI obs"""

        geo_projection = get_msg_projection()

        # Calculate x/y locations from cloudsat lat/lon
        cs_x, cs_y = geo_projection(
            self.cloudsat_ds.Longitude, self.cloudsat_ds.Latitude
        )

        # Calculate weighted average height of radar reflectivity
        filled_reflectivity = (
            np.clip(self.cloudsat_ds.Radar_Reflectivity, -35, 20).fillna(-35).values
        )
        average_reflectivity_height = np.average(
            self.cloudsat_ds.Height, weights=filled_reflectivity + 35 + 1e-15, axis=1
        )
        average_reflectivity_height[np.all(filled_reflectivity <= -35, axis=1)] = 0

        # Calculate SEVIRI viewing angles at cloudsat locations
        cloudsat_zenith_angle, cloudsat_azimuth_angle = get_satellite_viewing_angles(
            self.cloudsat_ds.Latitude, self.cloudsat_ds.Longitude
        )

        # Calculate parallax distance offset from reflectivity height and zenith angle
        parallax_dist = (
            np.sin(np.radians(cloudsat_zenith_angle)) * average_reflectivity_height
        )

        # Calculate x/y offsets due to parallax
        parallax_x = parallax_dist * np.sin(np.radians(cloudsat_azimuth_angle))
        parallax_y = parallax_dist * np.cos(np.radians(cloudsat_azimuth_angle))

        corrected_lon, corrected_lat = geo_projection(
            cs_x + parallax_x, cs_y + parallax_y, inverse=True
        )

        self.cloudsat_ds.Longitude.data = corrected_lon
        self.cloudsat_ds.Latitude.data = corrected_lat

In [24]:
# STEP 1: Create the MSG invariants file
create_msg_invariants()

In [26]:
# STEP 2: Load the MSG invariants file
MSG_INVARIANTS_PATH = "msg_invariants.nc"
msg_invariants = xr.open_dataset(MSG_INVARIANTS_PATH)
msg_invariants

<xarray.Dataset> Size: 441MB
Dimensions:                  (y: 3712, x: 3712)
Coordinates:
    acq_time                 (y) datetime64[ns] 30kB ...
  * y                        (y) float64 30kB -5.566e+06 ... 5.569e+06
  * x                        (x) float64 30kB 5.566e+06 5.563e+06 ... -5.569e+06
    longitude                (y, x) float64 110MB ...
    latitude                 (y, x) float64 110MB ...
Data variables:
    satellite_zenith_angle   (y, x) float64 110MB ...
    satellite_azimuth_angle  (y, x) float64 110MB ...
    time_offsets             (y) timedelta64[ns] 30kB ...
Attributes:
    history:      Created by pytroll/satpy on 2024-09-26 22:10:23.082740
    Conventions:  CF-1.7

In [ ]:
# NOTE: We would need to have access to the cloud top height in order to do the parallax correction, which we don't currently have.